In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import re
import random
import pandas as pd

In [22]:
# data = pd.read_csv('Textranked.csv')
# data.head()

# # data.dropna(inplace=True)
# # data.rename(columns={'본문':'question', '답변':'answer'}, inplace=True)
# # data.to_csv('prep_data_dotsplits.csv', index=False, encoding='utf-8')

In [23]:
SOS_token = 0
EOS_token = 1

class QNA:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split('.'):
            self.addWord(word)

    def addWord(self, word):
        for sen in word:
            if sen not in self.word2index:             
                self.word2index[sen] = self.n_words
                self.word2count[sen] = 1
                self.index2word[self.n_words] = sen
                self.n_words += 1
            else:
                self.word2count[sen] += 1

In [24]:
def normalizeString(s):
    s = re.sub(r"([.!?])", r" ", s)
    s = re.sub(r"[^가-힣.!?]+", r" ", s)
    return s

In [25]:
def readchat(question, answer, reverse=False):
    print("Reading lines...")

    lines = open('/content/drive/MyDrive/multi/202206/0609/data/prep_data_dotsplits.csv', encoding='utf-8').read().strip().split('\n')[1:]
    
    pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]

    if reverse:
        paris = [list(reversed(p)) for p in pairs]
        input_q = QNA(answer)
        output_a = QNA(question)
    else:
        input_q = QNA(question)
        output_a = QNA(answer)
    
    return input_q, output_a, pairs

In [26]:
MAX_LENGTH = 2000
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [27]:
def prepareData(question, answer, reverse=False):
    input_q, output_a, pairs = readchat(question, answer, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_q.addSentence(pair[0])
        output_a.addSentence(pair[1])
    print("Counted words:")
    print(input_q.name, input_q.n_words)
    print(output_a.name, output_a.n_words)
    return input_q, output_a, pairs

input_q, output_a, pairs = prepareData('question', 'answer', True)
print(random.choice(pairs))

Reading lines...
Read 18271 sentence pairs
Counted words:
answer 1657
question 1171
['전 세입자입니다 년에 만원에 세입자로 들어가서 현재 까지 살고 있습니다 그런데 제가 힘이들어서 세를 밀렸습니다 그래서 집주인께서 월말에 만원 월말까지 만원 을 지불안하면 방을 빼 깻다고 했습니다 그래서 월 일 날 두달분을 지불하고 월 일 까지 두달분을 지불하기로 했는데 직장에서 돈이 나오지가 않아서 월 말일날 지불했습니다 그런데 집주인께서 나가라고 합니다 현재 제가 밀려 있는 방세는 개월 정도 밀려 잇는데 월 말까지 집을 비우라고 합니다 그래서 제가 돈이 없어서 못나간다고 말했습니다 그랫더니만 월 달에 집주인 아저씨께서 집수리를 해야 하니 꼭옥 나가야 한다고 합니다 그래서 저는 못나간다고 말햇습니다 돈이 없다 라고영 그래더니만 집주인께서 명도 소송을 진행할거니깐 방 어더서 나가라고 합니다 오늘 은 집주인께서 법원에 지급 명령을 신청 하고 왓다고 문자가왔습니다 방 어드면 취소할테니 그럴게 알라고요 그리고 일방적으로 나가라고만 하는데 어떻게해야 하나요 저는 현재 통장 잔고나 부동산 같은게 하나도 없습니다 질문 제가 안나간다고 버티면 어떻게 되나요 명도소송에서 지게 되면 강제로 제짐을 뺄수 있는지용 그리고 나간다면 제가 이사 비용을 받을수있는지용 명도 소송해서 패소까지 기간이 얼마나 걸릴까요 현재 저는 수입이 없습니다 프리랜서 라서영 그런데 자꾸 나가 라고 하니 어떻게 해야 할지 모르겠습니다', '우선 결론부터 말씀드리면 집주인이 명도소송을 진행하여 승소하면 명도집행을 단행하게 됩니다 네 그렇습니다 귀하가 소송비용을 부담하게 됩니다 명도소송은 통상 개월 정도 소요됩니다 아래 내용은 채권자 입장에서 정리한 것입니다 채무자 관점에서 읽어보시면 나름대로의 해결방향을 찾을 수 있을 것입니다 채권추심에서 가장 중요한 가지는 정보력 신속 정확입니다 왜냐하면 다른 채권자들보다 먼저 재산을 확보해야만 자금을 회수할 수 있기 때문

In [47]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [29]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [30]:
def indexesFromSentence(QNA, sentence):
    word2index_list = []
    for sen in sentence.split('.'):
        for word in sen:
            word2index_list.append(QNA.word2index[word])
    return word2index_list


def tensorFromSentence(QNA, sentence):
    indexes = indexesFromSentence(QNA, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=DEVICE).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_q, pair[0])
    target_tensor = tensorFromSentence(output_a, pair[1])
    return (input_tensor, target_tensor)

In [59]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    if input_length >= max_length :
        input_length = max_length - 1

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=DEVICE)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [60]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [61]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [62]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [63]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_q, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=DEVICE)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_a.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [64]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_q.n_words, hidden_size).to(DEVICE)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_a.n_words, dropout_p=0.1).to(DEVICE)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [ ]:
encoder1.size(0)

In [ ]:
sentence2 = '직원과 사장으로 만나신분에게 대가성없이 통장을 대여해드렸습니다. 제 명의로 사업자를 내서 사업을 했으나 빛뿐이였습니다.'

In [ ]:
sentence1 = pairs[0][0]
sentence1

In [ ]:
input_q.word2index

In [ ]:
evaluate(encoder1, attn_decoder1, sentence1)

In [ ]:
evaluate(encoder1, attn_decoder1, sentence2)

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [ ]:
pairs